In [31]:
import pandas as pd
import os
import jellyfish


In [32]:
dataDir = r'C:\Users\Ian\Dropbox\ECON 407 - US Courts & Judges'


In [33]:
cm_data = pd.read_csv(os.path.join(dataDir,'Carp Manning Clean.csv'))
judge_data = pd.read_csv(os.path.join(dataDir,'Judge Attribute and Ideology.csv'))


In [34]:
# Delete this once notebooks are merged as it has already been defined.
def get_best_name_match_from_list(name, data_list):
    best_match = ""
    highest_jw = 0
    
    for potential_match in data_list:
        current_score = jellyfish.jaro_winkler(potential_match, name)
        if ((current_score > highest_jw) and (current_score > 0.89)):
            highest_jw = current_score
            best_match = potential_match
    
    return best_match

cm_data['Closest Name'] = cm_data['Authoring Judge'].apply(lambda x : get_best_name_match_from_list(x,judge_data['Name']))


In [35]:
cm_data = cm_data.merge(judge_data, left_on = 'Closest Name', right_on = 'Name', how = 'left')


In [36]:
cm_data_preserve = cm_data.copy()
duplicate_cols = list(filter(lambda col: col[-2:] == '_y', list(cm_data.columns)))
cm_data = cm_data.drop(columns = duplicate_cols)
cm_data = cm_data.drop(columns = ['Closest Name','Name'])
duplicate_cols = [col[:-2] for col in duplicate_cols]
for col in duplicate_cols:
    cm_data = cm_data.rename(columns = {col + '_x': col})


In [38]:
cm_data.to_csv(os.path.join(dataDir,'Carp Manning with Attributes.csv'),index = False)